In [ ]:
!pip install transformers
from transformers import AutoConfig, AutoModel, AutoTokenizer 
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
TASK=2
LANG='English'
if LANG=='Spanish' and TASK==1:
  TRANSFORMERMODEL='Roberta'
  MODELTYPE='large'
  CLASSIFIER='D'
  NCLASES=2
  DROPOUT=0.1
  MODELPATH='/content/drive/MyDrive/TFG_Exist2022/experimentos/bestmodelfull_task1_run1_es_POS2/20220331-070523'
  MODELNAME='task1esmodel.pt'
  PREPROCESS='N'
  THRESHOLD=0.5
  MAX_LEN = 128 
  BATCH_SIZE = 8
if LANG=='Spanish' and TASK==2:
  TRANSFORMERMODEL='Bert'
  MODELTYPE='base'
  CLASSIFIER='A'
  NCLASES=6
  DROPOUT=0.1
  MODELPATH='/content/drive/MyDrive/TFG_Exist2022/experimentos/bestmodelfull_task2_run1_es_POS12/20220331-130948'
  MODELNAME='task2esmodel.pt'
  PREPROCESS='N'
  THRESHOLD=0.5
  MAX_LEN = 200 
  BATCH_SIZE = 32
if LANG=='English' and TASK==1:
  TRANSFORMERMODEL='Roberta'
  MODELTYPE='large'
  CLASSIFIER='D'
  NCLASES=2
  DROPOUT=0.1
  MODELPATH='/content/drive/MyDrive/TFG_Exist2022/experimentos/bestmodelfull_task1_run1_en_POS12/20220330-101349'
  MODELNAME='task1enmodel.pt'
  PREPROCESS='N'
  THRESHOLD=0.5
  MAX_LEN = 128 
  BATCH_SIZE = 8
if LANG=='English' and TASK==2:
  TRANSFORMERMODEL='Roberta'
  MODELTYPE='large'
  CLASSIFIER='D'
  NCLASES=6
  DROPOUT=0.1
  MODELPATH='/content/drive/MyDrive/TFG_Exist2022/experimentos/bestmodelfull_task2_run1_en_POS12_task1_POS10/20220331-122913'
  MODELNAME='task2enmodel.pt'
  PREPROCESS='N'
  THRESHOLD=0.5
  MAX_LEN = 128
  BATCH_SIZE = 8

def load_checkpoint(load_path, model):
    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    model.load_state_dict(state_dict['model_state_dict'])

def get_tokenizer():
  if LANG == 'Spanish':
    if TRANSFORMERMODEL=='Bert': #BETO
      tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
    if TRANSFORMERMODEL == 'Roberta':
      tokenizer=AutoTokenizer.from_pretrained('xlm-roberta-large')
  if LANG == 'English':
    if TRANSFORMERMODEL=='Bert':
      if MODELTYPE == 'base': #large, xlarge, xxlarge
        tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
      if MODELTYPE == 'large': #large, xlarge, xxlarge
        tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')
    if TRANSFORMERMODEL == 'Roberta':
      if MODELTYPE == 'base': #large, xlarge, xxlarge
        tokenizer = AutoTokenizer.from_pretrained('roberta-base')
      if MODELTYPE == 'large': #large, xlarge, xxlarge
        tokenizer = AutoTokenizer.from_pretrained('roberta-large')
  return tokenizer

if TRANSFORMERMODEL=='Bert':
  class BertClassifier(nn.Module):
    def __init__(self, n_classes=2, dropout_rate=0.1):
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        H=50
        D_out=n_classes
        if LANG == 'English':
          if MODELTYPE == 'base': #large, xlarge, xxlarge
            D_in=768
            self.bert = AutoModel.from_pretrained('bert-base-uncased')
          if MODELTYPE == 'large': #large, xlarge, xxlarge
            D_in=1024
            self.bert = AutoModel.from_pretrained('bert-large-uncased')
        if LANG == 'Spanish':
          if MODELTYPE == 'base':
            D_in=768
            self.bert = AutoModel.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
          if MODELTYPE == 'basecased': #large, xlarge, xxlarge, basecased
            D_in=768
            self.bert = AutoModel.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
        if CLASSIFIER=='A':
          self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            nn.Linear(H, D_out)
          )
        if CLASSIFIER=='B':
          self.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(D_in,D_out),
            nn.Sigmoid()
          )
        if CLASSIFIER=='C':
          self.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(D_in,D_out)
          )
        if CLASSIFIER=='D':
          self.classifier = nn.Sequential(
            nn.Linear(D_in, D_in),
            nn.Dropout(dropout_rate),
            nn.Linear(D_in, D_out)
          )      
    def forward(self, input_ids, attention_mask):
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits
if TRANSFORMERMODEL=='Roberta':
  class ROBERTAClassifier(nn.Module):
    def __init__(self, n_classes=2, dropout_rate=0.1):
        super(ROBERTAClassifier, self).__init__()
        H=50
        D_out=n_classes
        if LANG == 'English':
          if MODELTYPE == 'base': #large, xlarge, xxlarge
            D_in=768
            self.roberta = AutoModel.from_pretrained('roberta-base') #,return_dict=False)
          if MODELTYPE == 'large': #large, xlarge, xxlarge
            D_in=1024
            self.roberta = AutoModel.from_pretrained('roberta-large') #,return_dict=False)
        if LANG == 'Spanish':
          if MODELTYPE == 'base': #large, xlarge, xxlarge
            D_in=768
            self.roberta = AutoModel.from_pretrained('xlm-roberta-base')
          if MODELTYPE == 'large': #large, xlarge, xxlarge
            D_in=1024
            self.roberta = AutoModel.from_pretrained('xlm-roberta-large')
        if CLASSIFIER=='A':
          self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            nn.Linear(H, D_out)
          )
        if CLASSIFIER=='D':
          self.classifier = nn.Sequential(
            nn.Linear(D_in, D_in),
            nn.Dropout(dropout_rate),
            nn.Linear(D_in, D_out)
          )
    def forward(self, input_ids, attention_mask):
        # Feed input to BERT
        outputs = self.roberta(input_ids=input_ids,attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

#################
#Set up GPU/CPU
#################
#Al igual que transfieres un Tensor a la GPU, transfieres la red neuronal a la GPU.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("cpu o gpu?:", device)
# Clearing Cache space for a fresh Model run
torch.cuda.empty_cache() 



#Instanciamos el modelo
if TRANSFORMERMODEL=='Bert':
  # Instantiate Bert Classifier
  model = BertClassifier(NCLASES,DROPOUT)
if TRANSFORMERMODEL=='Roberta':
  model=ROBERTAClassifier(NCLASES,DROPOUT)
#Instanciamos el tokenizador
tokenizer = get_tokenizer()
#Cargamos el modelo en la instancia
load_checkpoint(MODELPATH + '/' + MODELNAME, model)
#Ejecutamos el modelo en la GPU
model.to(device)
#non-sexist
sent="This is a beautitul life"
encoded_sent = tokenizer.encode_plus(
            text=sent,  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,                  # Max length to truncate/pad
            #return_token_type_ids=False, #return the token type IDs according to the specific tokenizer’s default
            pad_to_max_length=True,         # Pad sentence to max length
            return_attention_mask=True,      # Return attention mask
            return_tensors='pt',           # Return PyTorch tensor
            )

input_ids = encoded_sent['input_ids'].to(device)
attention_mask = encoded_sent['attention_mask'].to(device)
output = model(input_ids, attention_mask)
if NCLASES == 2:
  probs = F.softmax(output, dim=1).cpu()
  preds = (probs[:, 1] > THRESHOLD, 1, 0)
  predictiontensor= preds[0]
  prediction=predictiontensor.cpu().numpy()
  result=prediction[0]
  if result:
    print('sexist')
  else:
    print('non-sexist')
else:
  _,predicts=torch.max(output, dim=1)
  sexual_type=["ideological-inequality","misogyny-non-sexual-violence","non-sexist","objectification","sexual-violence","stereotyping-dominance"]
  predictiontensor=predicts[0]
  prediction=predictiontensor.cpu().numpy()
  result=sexual_type[prediction]
  print(result)
